In [1]:
import pandas as pd
from math import ceil

In [2]:
df = pd.read_csv("dataset.csv",index_col=0,parse_dates=[1],infer_datetime_format=True).dropna()

C:\Users\fai94s\AppData\Local\Temp\ipykernel_17768\2944890690.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv("dataset.csv",index_col=0,parse_dates=[1],infer_datetime_format=True).dropna()
C:\Users\fai94s\AppData\Local\Temp\ipykernel_17768\2944890690.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("dataset.csv",index_col=0,parse_dates=[1],infer_datetime_format=True).dropna()


In [3]:
df.dtypes

time           datetime64[ns]
seconds               float64
state                 float64
sensor_name            object
thing_name             object
thing_ip               object
dtype: object

In [4]:
df.head()

,time,seconds,state,sensor_name,thing_name,thing_ip
date,,,,,,
08/05/23,2024-09-05 15:26:00,1.691267e+09,84.92,Kitchen Temperature,KitchenPi,192.168.4.36
08/05/23,2024-09-05 15:26:00,1.691267e+09,56.70,Kitchen Humidity,KitchenPi,192.168.4.36
08/05/23,2024-09-05 15:27:00,1.691267e+09,50.00,Kitchen Humidity,KitchenPi,192.168.4.36
08/05/23,2024-09-05 15:27:00,1.691267e+09,84.92,Kitchen Temperature,KitchenPi,192.168.4.36
08/05/23,2024-09-05 15:28:00,1.691267e+09,50.00,Kitchen Humidity,KitchenPi,192.168.4.36


In [5]:
df.sensor_name.value_counts()

sensor_name
Desk Left Sonar                    448597
Desk Right Sonar                   198410
Motion Outside Room                 76114
Desk Left Motion                    73520
Motion Inside Room(West Corner)     63709
Entrance Motion                     61423
Motion Inside Room(East Corner)     61200
Kitchen Humidity                    48763
Kitchen Temperature                 48763
Washroom Temperature                45938
Washroom Humidity                   45938
Bedroom Humidity                    40676
Bedroom Temperature                 40676
Desk Right Motion                   26088
Desk Left Light                      3242
Kitchen Light                         697
Room Door                             517
Entrance Door                         400
Closet Door                           226
Closet Light                           74
Name: count, dtype: int64

In [ ]:
df.sensor_name.value_counts().to_csv('value_counts.csv')

: 

In [7]:
df.sort_values(["seconds"])

,time,seconds,state,sensor_name,thing_name,thing_ip
date,,,,,,
08/05/23,2023-10-06 15:26:00,1.691267e+09,84.92,Kitchen Temperature,KitchenPi,192.168.4.36
08/05/23,2023-10-06 15:26:00,1.691267e+09,56.70,Kitchen Humidity,KitchenPi,192.168.4.36
08/05/23,2023-10-06 15:27:00,1.691267e+09,50.00,Kitchen Humidity,KitchenPi,192.168.4.36
08/05/23,2023-10-06 15:27:00,1.691267e+09,84.92,Kitchen Temperature,KitchenPi,192.168.4.36
08/05/23,2023-10-06 15:28:00,1.691267e+09,50.00,Kitchen Humidity,KitchenPi,192.168.4.36
...,...,...,...,...,...,...
09/15/23,2023-10-06 12:15:00,1.694798e+09,80.96,Washroom Temperature,WashroomPi,192.168.4.37
09/15/23,2023-10-06 12:15:00,1.694798e+09,44.40,Washroom Humidity,WashroomPi,192.168.4.37
09/15/23,2023-10-06 12:15:00,1.694798e+09,79.70,Bedroom Temperature,BedroomTemperaturePi,192.168.4.38


In [ ]:
#run the following 3 cells
u = df.sensor_name.unique()
adjMat = pd.DataFrame(0.0, columns=u, index=u)
adjMat2 = pd.DataFrame(0.0, columns=u, index=u)


In [ ]:
# fne
prevRow = df.iloc[0]
for _,curRow in df.iterrows():
    if (curRow.sensor_name != prevRow.sensor_name):
        t = max(ceil(curRow.seconds - prevRow.seconds), 1.0)
        # t = max(ceil((curRow.DateTime - prevRow.DateTime).total_seconds()),1.0)
        adjMat[prevRow.sensor_name][curRow.sensor_name] += 1/t
    prevRow = curRow

In [ ]:
#td
curGroup = []
th = 2
prevRow = df.iloc[0]
for _, curRow in df.iterrows():
    if ((curRow.seconds - prevRow.seconds) >= th):
        curGroup = []
    if not(curRow.sensor_name in curGroup):
        curGroup.append(curRow.sensor_name)
    for item in curGroup:
        adjMat2[item][curRow.sensor_name] += 1
    prevRow = curRow

In [20]:
gb = df[["DateTime","Sensor"]].groupby([df.DateTime.dt.minute, df.Sensor])
newdf = gb.count()
newdf = newdf.rename(columns={"DateTime": "Value"}).reset_index(level=0)
pivoted = newdf.pivot(columns="DateTime").fillna(0)
pivoted.columns = pivoted.columns.droplevel(0)

In [11]:
adjMat.to_csv("FNEadjMat_09_15.csv")


In [12]:
adjMat2.to_csv("TD5adjMat_09_15.csv")

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
freqMat = pivoted
freqMat = freqMat.div(freqMat.sum(axis=1), axis=0)
cs = pd.DataFrame(cosine_similarity(freqMat),
                    index=freqMat.index, columns=freqMat.index).round(2)

In [8]:
def normalize(df_adj):
    r=0
    norm = df_adj.copy()
    for index, row in norm.iterrows():
        mx = 0
        mn = 0
        c = 0
        for value in row:
            mx = max(mx, value)
            mn = min(mn, value)
            if not mx-mn == 0:
                norm.iloc[r,c] = (value-mn)/(mx-mn)
            c += 1
        r += 1
    return norm


In [9]:
norm_adjMat = normalize(adjMat)
norm_adjMat

,Kitchen Temperature,Kitchen Humidity,Bedroom Temperature,Bedroom Humidity,Motion Inside Room(West Corner),Washroom Humidity,Washroom Temperature,Entrance Motion,Entrance Door,Motion Outside Room,Motion Inside Room(East Corner),Room Door,Closet Door,Closet Light,Desk Right Sonar,Desk Right Motion,Desk Left Motion,Desk Left Sonar,Desk Left Light,Kitchen Light
Kitchen Temperature,0.0,1.000000,0.011959,0.007938,0.019249,0.018542,0.018591,0.019675,0.000083,0.020405,0.010387,0.000127,0.000045,0.000097,0.068387,0.006297,0.030654,0.122871,0.000892,0.000324
Kitchen Humidity,1.0,0.000000,0.010911,0.007237,0.020004,0.018054,0.017129,0.020178,0.000109,0.021172,0.010264,0.000024,0.000000,0.000000,0.064129,0.006648,0.030428,0.117740,0.000619,0.000271
Bedroom Temperature,1.0,1.000000,0.000000,1.000000,0.721186,0.017108,0.018328,0.017658,0.000124,0.021643,0.008951,0.000068,0.000000,0.000000,0.075255,0.006176,0.034460,0.127392,0.001143,0.000115
Bedroom Humidity,1.0,0.864011,1.000000,0.000000,0.981282,0.012262,0.011247,0.012719,0.000013,0.016288,0.007112,0.000132,0.000099,0.000007,0.049200,0.005405,0.022759,0.086127,0.000588,0.000022
Motion Inside Room(West Corner),1.0,1.000000,1.000000,1.000000,0.000000,0.027099,0.028685,0.023767,0.000254,0.120988,0.255487,0.004888,0.003173,0.000944,0.143454,0.017099,0.069948,0.254115,0.000774,0.000095
Washroom Humidity,1.0,0.977722,0.552752,0.347232,0.942764,0.000000,1.000000,0.020665,0.000051,0.022730,0.011227,0.000000,0.000022,0.000065,0.068177,0.006614,0.030445,0.120949,0.000953,0.000232
Washroom Temperature,1.0,0.980949,0.534569,0.378669,0.951889,1.000000,0.000000,0.019037,0.000114,0.022607,0.010444,0.000087,0.000107,0.000022,0.069729,0.006578,0.030132,0.122984,0.000812,0.000266
Entrance Motion,1.0,0.927925,0.495893,0.380981,0.768188,0.891654,0.910760,0.000000,0.422839,1.000000,0.103958,0.000180,0.000241,0.000120,0.197514,0.024207,0.069161,0.298948,0.002677,0.007605
Entrance Door,1.0,0.445301,0.089644,0.373342,0.781838,0.982146,1.000000,1.000000,0.000000,0.288864,0.070874,0.000000,0.000000,0.000000,0.083937,0.031141,0.060135,0.187626,0.009665,0.012886
Motion Outside Room,1.0,1.000000,0.524497,0.388088,1.000000,0.260479,0.285092,1.000000,0.001925,0.000000,0.382549,0.006124,0.001336,0.000100,0.221047,0.108285,0.195900,0.356170,0.002571,0.012798


In [21]:
norm_adjMat = normalize(adjMat)

In [10]:
from sklearn.cluster import SpectralClustering

In [ ]:
#Run this
sp = SpectralClustering(3,affinity="precomputed")
clus = sp.fit_predict(norm_adjMat)

c:\Users\fai94s\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_spectral_embedding.py:248: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


In [15]:
sp_2 = SpectralClustering(5,affinity="precomputed")
clus_2 = sp_2.fit_predict(norm_adjMat2)

/Users/irfan/.local/share/virtualenvs/CSC798-CD72dcE_/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:247: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


In [12]:
display(pd.DataFrame(clus,index=adjMat.index))

,0
Kitchen Temperature,1
Kitchen Humidity,0
Bedroom Temperature,0
Bedroom Humidity,0
Motion Inside Room(West Corner),0
Washroom Humidity,1
Washroom Temperature,1
Entrance Motion,1
Entrance Door,1
Motion Outside Room,2


In [13]:
# display(pd.DataFrame(clus_2,index=adjMat2.index))
td_cluster = pd.DataFrame(clus,index=adjMat.index)
cluster_dict = {}
for index, row in td_cluster.iterrows():
    for value in row:
        # print(value, index)
        if not value in cluster_dict:
            cluster_dict[value] = []
        cluster_dict[value].append(index)
for item in cluster_dict:
    print(cluster_dict[item])
# print(cluster_dict)

['Kitchen Temperature', 'Washroom Humidity', 'Washroom Temperature', 'Entrance Motion', 'Entrance Door', 'Desk Left Light', 'Kitchen Light']
['Kitchen Humidity', 'Bedroom Temperature', 'Bedroom Humidity', 'Motion Inside Room(West Corner)', 'Motion Inside Room(East Corner)', 'Room Door', 'Closet Door', 'Closet Light']
['Motion Outside Room', 'Desk Right Sonar', 'Desk Right Motion', 'Desk Left Motion', 'Desk Left Sonar']


In [14]:
list_adjs = []
list_unique_sensors = []
for key in cluster_dict:
    adj = pd.DataFrame(0.0, columns=cluster_dict[key], index=cluster_dict[key])
    list_unique_sensors.append(cluster_dict[key])
    for index in cluster_dict[key]:
        for head in cluster_dict[key]:
            adj.loc[index,head] = adjMat.loc[index,head]
    display(adj)
    list_adjs.append(adj)

# adjMat2.loc['Kitchen Temperature','Kitchen Temperature']


,Kitchen Temperature,Washroom Humidity,Washroom Temperature,Entrance Motion,Entrance Door,Desk Left Light,Kitchen Light
Kitchen Temperature,0.000000,447.799210,448.971772,475.163106,2.000000,21.548973,7.830357
Washroom Humidity,458.811392,0.000000,23035.356620,476.018513,1.166667,21.952036,5.333333
Washroom Temperature,468.012259,22903.521564,0.000000,436.022888,2.619438,18.587008,6.086250
Entrance Motion,504.300609,449.661836,459.296746,0.000000,213.238095,44.500000,126.402413
Entrance Door,2.956944,2.904152,3.044177,155.205556,0.000000,1.500000,2.000000
Desk Left Light,15.153927,19.860317,19.323492,40.458333,1.500000,0.000000,4.000000
Kitchen Light,6.477778,4.871795,3.025848,106.285354,2.500000,4.000000,0.000000


,Kitchen Humidity,Bedroom Temperature,Bedroom Humidity,Motion Inside Room(West Corner),Motion Inside Room(East Corner),Room Door,Closet Door,Closet Light
Kitchen Humidity,0.000000,268.561939,178.137270,492.397641,252.647784,0.583333,0.000000,0.000000
Bedroom Temperature,310.571829,0.000000,14757.142229,10642.638584,132.088863,1.000000,0.000000,0.000000
Bedroom Humidity,175.263090,15166.108907,0.000000,14882.231926,107.864637,2.000000,1.500000,0.100000
Motion Inside Room(West Corner),457.978208,10678.622367,15717.433970,0.000000,4015.606062,76.833333,49.875000,14.833333
Motion Inside Room(East Corner),238.706108,125.639611,91.667037,4472.043304,0.000000,148.833333,70.009524,11.066667
Room Door,4.114583,2.040000,1.250000,59.033333,118.583333,0.000000,0.000000,0.000000
Closet Door,0.250000,0.000000,1.000000,41.333333,95.083333,0.000000,0.000000,0.250000
Closet Light,0.342857,0.200000,0.000000,10.176190,11.216667,0.000000,0.750000,0.000000


,Motion Outside Room,Desk Right Sonar,Desk Right Motion,Desk Left Motion,Desk Left Sonar
Motion Outside Room,0.000000,3846.845897,1884.457215,3409.206790,6198.364723
Desk Right Sonar,3755.987428,0.000000,2832.503080,5687.045598,27217.547527
Desk Right Motion,1637.618056,1832.517443,0.000000,6846.541388,2580.809875
Desk Left Motion,3598.648974,6295.381976,6029.565032,0.000000,12931.563413
Desk Left Sonar,6034.287329,27515.680167,2331.902715,14150.250755,0.000000


In [17]:
adjMat2.to_csv("minuteAdjMatModified.csv")

In [15]:
pd.DataFrame(clus,index=adjMat.index).to_csv("FNEClusters_09_15.csv")

In [30]:
pd.DataFrame(clus,index=adjMat2.index).to_csv("TD5Clusters_09_15.csv")

In [42]:
print(adjMat2.iloc[0])

Kitchen Temperature                48763.0
Kitchen Humidity                   24149.0
Bedroom Temperature                  772.0
Bedroom Humidity                     772.0
Motion Inside Room(West Corner)     1176.0
Washroom Humidity                    856.0
Washroom Temperature                 856.0
Entrance Motion                     1158.0
Entrance Door                         16.0
Motion Outside Room                 1426.0
Motion Inside Room(East Corner)      617.0
Room Door                             12.0
Closet Door                            3.0
Closet Light                           4.0
Desk Right Sonar                    2731.0
Desk Right Motion                    500.0
Desk Left Motion                    1550.0
Desk Left Sonar                     4454.0
Desk Left Light                       48.0
Kitchen Light                         15.0
Name: Kitchen Temperature, dtype: float64


In [16]:
unique_sensors = []
for i in adjMat.index:
    unique_sensors.append(i)

In [17]:
def sensor_grouping(adjMat, number_of_sensors, unique_sensors):
    sensor_groups = []

    for index,row in adjMat.iterrows():
        initial_sensor = 0
        sensor_value_dict = {}
        for value in row:
            sensor_value_dict[unique_sensors[initial_sensor]] = value
            initial_sensor += 1
        
        sorted_dict = dict(sorted(sensor_value_dict.items(), reverse=True, key=lambda x: x[1]))
        # print(index)
        # print(sorted_dict)
        cnt = 0
        group = []
        group.append(index)
        for key in sorted_dict.keys():
            # print(key)
            if key != index:
                cnt+=1
                group.append(key)
            if cnt>=number_of_sensors-1:
                break
        group = sorted(group)
        sensor_groups.append(group)

    # print(sensor_groups)
        
    mySet = set()

    for j in range(len(sensor_groups)):
        mySet = mySet | set([tuple(sensor_groups[j])])

    print('number of unique group ', len(mySet))
    for item in mySet:
        print(item)
    
    # sensor_group_list = []
    # for item in mySet:
    #     group_list = []
    #     for element in item:
    #         group_list.append(element)
    #     sensor_group_list.append(group_list)
    # print(sensor_group_list)
    # merged_groups = []
    # for i in range(len(sensor_group_list)):
    #     # first_sensor = sensor_group_list[i][0]
    #     curr_group = sensor_group_list[i].copy()
    #     flag = True
    #     for j in range(len(sensor_group_list) - 1):
    #         comp_group = sensor_group_list[j].copy()
    #         ind_group = curr_group.copy()
    #         for item in comp_group:
    #             if not item in ind_group:
    #                 ind_group.append(item)
    #         if len(ind_group) == 4:
    #             merged_groups.append(sorted(ind_group))
    #             flag = False
    #     if flag and not curr_group in merged_groups:
    #         merged_groups.append(sorted(curr_group))


    # print(sensor_group_list)
    # print('group in main list: ', len(sensor_group_list))
    # for item in sensor_group_list:
        # print(item)
    # print(merged_groups)
    # mergedSet = set()

    # for j in range(len(merged_groups)):
    #     mergedSet = mergedSet | set([tuple(merged_groups[j])])
    # print('group in merged list: ', len(mergedSet))
    # for item in mergedSet:
    #     print(item)


# sensor_grouping(adjMat2, 3, unique_sensors)

In [18]:
i = 0

for item in list_adjs:
    sensor_grouping(list_adjs[i], 3 ,list_unique_sensors[i])
    i+=1

number of unique group  6
('Desk Left Light', 'Entrance Motion', 'Washroom Humidity')
('Entrance Motion', 'Kitchen Light', 'Kitchen Temperature')
('Entrance Motion', 'Kitchen Temperature', 'Washroom Temperature')
('Entrance Door', 'Entrance Motion', 'Washroom Temperature')
('Entrance Motion', 'Washroom Humidity', 'Washroom Temperature')
('Kitchen Temperature', 'Washroom Humidity', 'Washroom Temperature')
number of unique group  6
('Bedroom Temperature', 'Kitchen Humidity', 'Motion Inside Room(West Corner)')
('Bedroom Humidity', 'Bedroom Temperature', 'Motion Inside Room(West Corner)')
('Closet Door', 'Motion Inside Room(East Corner)', 'Motion Inside Room(West Corner)')
('Closet Light', 'Motion Inside Room(East Corner)', 'Motion Inside Room(West Corner)')
('Motion Inside Room(East Corner)', 'Motion Inside Room(West Corner)', 'Room Door')
('Kitchen Humidity', 'Motion Inside Room(East Corner)', 'Motion Inside Room(West Corner)')
number of unique group  3
('Desk Left Sonar', 'Desk Right So

In [28]:
sensor_grouping(adjMat2, 4, unique_sensors)

number of unique group  14
('Desk Left Sonar', 'Desk Right Sonar', 'Kitchen Humidity', 'Kitchen Temperature')
('Desk Left Sonar', 'Entrance Door', 'Entrance Motion', 'Motion Outside Room')
('Closet Door', 'Motion Inside Room(East Corner)', 'Motion Inside Room(West Corner)', 'Motion Outside Room')
('Desk Left Motion', 'Desk Left Sonar', 'Desk Right Sonar', 'Motion Inside Room(West Corner)')
('Desk Left Motion', 'Desk Left Sonar', 'Entrance Motion', 'Motion Outside Room')
('Desk Left Light', 'Desk Left Motion', 'Desk Left Sonar', 'Desk Right Sonar')
('Closet Light', 'Motion Inside Room(East Corner)', 'Motion Inside Room(West Corner)', 'Motion Outside Room')
('Motion Inside Room(East Corner)', 'Motion Inside Room(West Corner)', 'Motion Outside Room', 'Room Door')
('Desk Left Sonar', 'Entrance Motion', 'Kitchen Light', 'Motion Outside Room')
('Desk Left Sonar', 'Desk Right Sonar', 'Entrance Motion', 'Motion Outside Room')
('Entrance Motion', 'Motion Inside Room(East Corner)', 'Motion Insid

In [76]:
def sensor_grouping_fne(adjMat, number_of_sensors, unique_sensors):
    sensor_groups = []

    for index,row in adjMat.iterrows():
        initial_sensor = 0
        sensor_value_dict = {}
        for value in row:
            sensor_value_dict[unique_sensors[initial_sensor]] = value
            initial_sensor += 1
        
        sorted_dict = dict(sorted(sensor_value_dict.items(), reverse=True, key=lambda x: x[1]))
        # print(index)
        # print(sorted_dict)
        cnt = 0
        group = []
        group.append(index)
        for key in sorted_dict.keys():
            # print(key)
            cnt+=1
            group.append(key)
            if cnt>=number_of_sensors-1:
                break
        group = sorted(group)
        sensor_groups.append(group)
    mySet = set()

    for j in range(len(sensor_groups)):
        mySet = mySet | set([tuple(sensor_groups[j])])

    print('number of unique group ', len(mySet))
    for item in mySet:
        print(item)

sensor_grouping_fne(adjMat, 3, unique_sensors)

number of unique group  14
('Entrance Motion', 'Motion Inside Room(East Corner)', 'Motion Outside Room')
('Desk Left Motion', 'Desk Left Sonar', 'Desk Right Sonar')
('Desk Left Light', 'Desk Left Sonar', 'Desk Right Sonar')
('Closet Door', 'Motion Inside Room(East Corner)', 'Motion Inside Room(West Corner)')
('Entrance Door', 'Entrance Motion', 'Motion Outside Room')
('Bedroom Humidity', 'Bedroom Temperature', 'Motion Inside Room(West Corner)')
('Motion Inside Room(East Corner)', 'Motion Outside Room', 'Room Door')
('Motion Inside Room(East Corner)', 'Motion Inside Room(West Corner)', 'Motion Outside Room')
('Desk Left Sonar', 'Kitchen Humidity', 'Kitchen Temperature')
('Desk Left Sonar', 'Washroom Humidity', 'Washroom Temperature')
('Closet Light', 'Motion Inside Room(East Corner)', 'Motion Inside Room(West Corner)')
('Desk Left Motion', 'Desk Left Sonar', 'Desk Right Motion')
('Entrance Motion', 'Kitchen Light', 'Motion Outside Room')
('Desk Left Sonar', 'Entrance Motion', 'Motion Ou